## Retrieve 3D Reflectivity from WRF Simulations using WRF-Python. 

**For [High Resolution WRF Simulations of the Current and Future Climate of North America](https://rda.ucar.edu/datasets/ds612.0/).**

**Hungjui Yu 20210817**

In [51]:
%reset
import sys
import time
import datetime as dt
import pytz
from netCDF4 import Dataset
import numpy as np
import xarray as xr
import pandas as pd
import wrf
from wrf import (getvar, dbz, extract_times, destagger)
# import metpy
# import metpy.calc as mpcalc
# import metpy.units as units
# print(metpy.__version__)

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


**Set input files paths and names:**

In [52]:
def set_input_names(file_date):

    file_path_1 = '/gpfs/fs1/collections/rda/data/ds612.0'
    file_path_2 = '/CTRL3D'
    file_path_3 = '/{}'.format(file_date.strftime('%Y'))

    file_names = dict( P = file_path_1 + file_path_2 + file_path_3 + '/wrf3d_d01_CTRL_P_{}.nc'.format(file_date.strftime('%Y%m%d')) \
                     , TK = file_path_1 + file_path_2 + file_path_3 + '/wrf3d_d01_CTRL_TK_{}.nc'.format(file_date.strftime('%Y%m%d')) \
                     , QVAPOR = file_path_1 + file_path_2 + file_path_3 + '/wrf3d_d01_CTRL_QVAPOR_{}.nc'.format(file_date.strftime('%Y%m%d')) \
                     , QRAIN = file_path_1 + file_path_2 + file_path_3 + '/wrf3d_d01_CTRL_QRAIN_{}.nc'.format(file_date.strftime('%Y%m')) \
                     , QSNOW = file_path_1 + file_path_2 + file_path_3 + '/wrf3d_d01_CTRL_QSNOW_{}.nc'.format(file_date.strftime('%Y%m%d')) \
                     , QGRAUP = file_path_1 + file_path_2 + file_path_3 + '/wrf3d_d01_CTRL_QGRAUP_{}.nc'.format(file_date.strftime('%Y%m')) \
                     , Z = file_path_1 + file_path_2 + file_path_3 + '/wrf3d_d01_CTRL_Z_{}.nc'.format(file_date.strftime('%Y%m%d')) \
                     )
    
    return file_names

# file_name_list = set_file_paths_names(file_date_time)
# print(file_name_list)

**Get wrf output variables:**

In [53]:
def get_wrf_vars(file_name, var_name, time_index):

    wrf_file = Dataset(file_name)
    # wrf_var = getvar(wrf_file, wrf_var_to_read, timeidx=time_index_1) # This doesn't work for CONUS run files.
    wrf_var = getvar(wrf_file, var_name, timeidx=time_index, meta=False)
    # wrf_var_time = wrf.extract_times(wrf_file, timeidx=time_index)
    # print(wrf_var_time)
    
    return wrf_var

**Calculation for dBZ:**

In [54]:
def calculate_wrf_dbz(wrf_pres, wrf_temp, wrf_qv, wrf_qr, wrf_qs, wrf_qg):
    
    wrf_dbz = dbz(wrf_pres \
                , wrf_temp \
                , wrf_qv \
                , wrf_qr \
                , wrf_qs \
                , wrf_qg \
                # , use_varint=True \
                , use_liqskin=False \
                , meta=True \
                 )
    
    return wrf_dbz

**Set output file path and name:**

In [55]:
def set_output_name(file_date_time):

    # output_path = '/glade/u/home/hungjui/2scratch/DATA_WRF_CONUS_1_dBZ_v1.0'
    
    output_time = pd.to_datetime(wrf_3hour_list_1[hi]).strftime('%Y%m%d%H') # If input time type is numpy.datetime64:
    
    # output_name = output_path + '/wrf3d_d01_dbz_{}.nc'.format(file_date_time.strftime('%Y%m%d%H'))
    # output_name = output_path + '/wrf3d_d01_dbz_{}.nc'.format(output_time)
    output_name = '/wrf3d_d01_dbz_{}.nc'.format(output_time)

    return output_name


### Main Program:

In [56]:
start = time.time()

## WRF Model Simulation Category:
wrf_sim_type = 'CTRL3D'

## Loop through a period:
target_date_range = pd.date_range(start='2013-01-01', end='2013-6-30', tz=pytz.utc)

for tdi in range(len(target_date_range)):

    ## Set file datetime:
    # file_date_time = dt.datetime(2013, 9, 13, 0, 0, 0, tzinfo=pytz.utc)
    file_date_time = target_date_range[tdi]
    print('Processing: {}'.format(file_date_time.strftime('%Y%m%d')))
    
    ## Set input files paths and names:
    file_name_dict = set_input_names(file_date_time)

    ## Get the 3-hourly time list from P and QRAIN files:
    wrf_3hour_list_1 = wrf.extract_times(Dataset(file_name_dict['P']), timeidx=wrf.ALL_TIMES, meta=False, do_xtime=False)
    wrf_3hour_list_2 = wrf.extract_times(Dataset(file_name_dict['QRAIN']), timeidx=wrf.ALL_TIMES, meta=False, do_xtime=False)

    ## Set wrf variable list for reflectivity retrieval:
    wrf_vars_list = ['P', 'TK', 'QVAPOR', 'QRAIN', 'QSNOW', 'QGRAUP']

    for hi in range(len(wrf_3hour_list_1)):

        ## Get the index for common time in different files (every 3-hour):
        common_index_2 = np.intersect1d(wrf_3hour_list_1[hi], wrf_3hour_list_2, return_indices=True)[2][0]

        ## Get wrf output variables:
        wrf_vars = {}
        for vname in wrf_vars_list:

            file_name = file_name_dict[vname]

            if ( vname in ['QRAIN', 'QGRAUP'] ):
                wrf_vars['{}'.format(vname)] = get_wrf_vars(file_name, vname, common_index_2)
                # wrf_vars['{}'.format(vname)] = xr.open_dataset(file_name)
            #elif ()
            else:
                wrf_vars['{}'.format(vname)] = get_wrf_vars(file_name, vname, hi)
                # wrf_vars['{}'.format(vname)] = xr.open_dataset(file_name)


        ## Calculation for dBZ:
        wrf_dbz = calculate_wrf_dbz(wrf_vars['P'],
                                    wrf_vars['TK'], 
                                    wrf_vars['QVAPOR'],
                                    wrf_vars['QRAIN'],
                                    wrf_vars['QSNOW'],
                                    wrf_vars['QGRAUP']
                                    ) # .to_dataset()

        ## Unstagger Z vertical grids:
        # wrf_dataset_Z = Dataset(file_name_dict['Z'])
        # wrf_var_Z = getvar(Dataset(file_name_dict['Z']), 'Z', timeidx=hi, meta=False)
        wrf_var_Z_unstag = wrf.destagger(getvar(Dataset(file_name_dict['Z']), 'Z', timeidx=hi, meta=False), 0)

        ## Get AGL:
        # wrf_var_Z_AGL = wrf.g_geoht.get_height_agl(Dataset(file_name_dict['Z']), 'Z', meta=False)

        ## Add Z and dBZ into dataset of P:
        wrf_dataset_P_Z_dBZ = xr.open_dataset(file_name_dict['P']).isel(Time = hi)
        wrf_dataset_P_Z_dBZ['TK'] = (['bottom_top', 'south_north', 'west_east'], wrf_vars['TK'])
        wrf_dataset_P_Z_dBZ['Z'] = (['bottom_top', 'south_north', 'west_east'], wrf_var_Z_unstag)
        wrf_dataset_P_Z_dBZ['dBZ'] = (['bottom_top', 'south_north', 'west_east'], wrf_dbz)

        ## Calculate Z using the U.S. standard atmosphere & Hypsometric eqn.:
        # Z_standard = mpcalc.pressure_to_height_std((wrf_dataset_P_Z_dBZ['P'].values) * units.units.Pa)
        # wrf_dataset_P_Z_dBZ['Z_standard'] = (['bottom_top', 'south_north', 'west_east'], Z_standard)

        ## Set coordinates and dimensions:

        ## Set output file path and name:
        output_path_1 = '/glade/u/home/hungjui/2scratch/DATA_WRF_CONUS_1_dBZ_v1.0/' + wrf_sim_type
        # output_path_2 = '/20130913'
        output_path_2 = '/{}'.format(file_date_time.strftime('%Y'))
        output_file_name = set_output_name(wrf_3hour_list_1[hi])
        wrf_dataset_P_Z_dBZ.to_netcdf(output_path_1 + output_path_2 + output_file_name)

end = time.time()
print("RUNTIME：%f SEC" % (end - start))
print("RUNTIME：%f HOUR" % ((end - start)/3600))

Processing: 20130101
Processing: 20130102
Processing: 20130103
Processing: 20130104
Processing: 20130105
Processing: 20130106
Processing: 20130107
Processing: 20130108
Processing: 20130109
Processing: 20130110
Processing: 20130111
Processing: 20130112
Processing: 20130113
Processing: 20130114
Processing: 20130115
Processing: 20130116
Processing: 20130117
Processing: 20130118
Processing: 20130119
Processing: 20130120
Processing: 20130121
Processing: 20130122
Processing: 20130123
Processing: 20130124
Processing: 20130125
Processing: 20130126
Processing: 20130127
Processing: 20130128
Processing: 20130129
Processing: 20130130
Processing: 20130131
Processing: 20130201
Processing: 20130202
Processing: 20130203
Processing: 20130204
Processing: 20130205
Processing: 20130206
Processing: 20130207
Processing: 20130208
Processing: 20130209
Processing: 20130210
Processing: 20130211
Processing: 20130212
Processing: 20130213
Processing: 20130214
Processing: 20130215
Processing: 20130216
Processing: 2